#### TODO：尝试拆分句子为短句

In [1]:
import fasttext
import os
import json
import re
from utils import fixText, resetLabelLv2, resetLabelLv3
from model import MyFastText

TEMP_DIR = './tmp/'

def getConfig():
    config = {}
    with open('./config.json', 'r') as f:
        s = f.read()
        config = json.loads(s)
    return config

def split_text(text):
    splited_text = re.split(r'\n+|\r+|\r\n|<br>', text)
    while '' in splited_text:
        splited_text.remove('')
    return splited_text

config = getConfig()

In [2]:
# 读入数据集的json文件，处理成fasttext接口使用的"文本__label__标签"形式，以txt文件存储
def readDataSet(config):
    set1 = []
    with open(config['data_path'] + config['dataset_label_1_name'], 'r') as f:
        s = f.read()
        data_set = json.loads(s)
        for id in list(data_set.keys()):
            text = data_set[id]['text']
            splited_text = split_text(text)
            for text in splited_text:
                set1.append('__label__' + data_set[id]['label_level_1'] + ' ' + fixText(text))
    set2 = []
    with open(config['data_path'] + config['dataset_label_2_name'], 'r') as f:
        s = f.read()
        data_set = json.loads(s)
        for id in list(data_set.keys()):
            text = data_set[id]['text']
            splited_text = split_text(text)
            for text in splited_text:
                set2.append('__label__' + data_set[id]['label_level_2'] + ' ' + fixText(text))
    set3 = []
    with open(config['data_path'] + config['dataset_label_3_name'], 'r') as f:
        s = f.read()
        data_set = json.loads(s)
        for id in list(data_set.keys()):
            text = data_set[id]['text']
            splited_text = split_text(text)
            for text in splited_text:
                set3.append('__label__' + data_set[id]['label_level_3'] + ' ' + fixText(text))
        
    try:
        os.mkdir(TEMP_DIR)
    except:
        pass
    with open(TEMP_DIR + 'set1.txt', 'w') as f:
        print('训练集1拆分后共{}条文本'.format(len(set1)))
        for l in set1:
            f.write(l + '\n')
    with open(TEMP_DIR + 'set2.txt', 'w') as f:
        print('训练集2拆分后共{}条文本'.format(len(set2)))
        for l in set2:
            f.write(l + '\n')
    with open(TEMP_DIR + 'set3.txt', 'w') as f:
        print('训练集3拆分后共{}条文本'.format(len(set2)))
        for l in set3:
            f.write(l + '\n')

readDataSet(config)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.785 seconds.
Prefix dict has been built successfully.


训练集2拆分后共1285245条文本


In [3]:
model_label_1 = MyFastText()
model_label_1.train(
    input = TEMP_DIR + 'set1.txt',
    lr = config['lr'],
    dim = config['hidden_dim'],
    epoch = config['epoch'],
)
model_label_1.save(config['model_path'] + config['model_label_1_name'])

Read 2M words
Number of words:  61624
Number of labels: 85
Progress: 100.0% words/sec/thread:  193551 lr:  0.000000 avg.loss:  1.508976 ETA:   0h 0m 0s


In [3]:
model_label_2 = MyFastText()
model_label_2.train(
    input = TEMP_DIR + 'set2.txt',
    lr = config['lr'],
    dim = config['hidden_dim'],
    epoch = config['epoch']
)
model_label_2.save(config['model_path'] + config['model_label_2_name'])

Read 20M words
Number of words:  233142
Number of labels: 453
Progress:   1.1% words/sec/thread:   37475 lr:  0.098895 avg.loss:  2.244689 ETA:   0h31m31s

In [ ]:
model_label_3 = MyFastText()
model_label_3.train(
    input = TEMP_DIR + 'set3.txt',
    lr = config['lr'],
    dim = config['hidden_dim'],
    epoch = config['epoch']
)
model_label_3.save(config['model_path'] + config['model_label_2_name'])

In [9]:
try:
    os.remove(TEMP_DIR + 'set1.txt')
    os.remove(TEMP_DIR + 'set2.txt')
    os.remove(TEMP_DIR + 'set3.txt')
    os.removedirs(TEMP_DIR)
except Exception as e:
    print(e)

[Errno 2] No such file or directory: './tmp/set1.txt'


In [4]:
model_label1 = MyFastText()
model_label1.load_model('./model/model_for_label_1.model')

In [5]:
with open(config['data_path'] + 'test_set.json', 'r') as f:
    content = json.loads(f.read())

total = 0
true_label_1 = 0
true_label_2 = 0
true_label_3 = 0

# label_1_dict = {}
label_2_dict = {}
for id in content.keys():
    text = content[id]['text']
    
    # label_1 = content[id]['label_level_1']
    label_2 = resetLabelLv2(content[id]['label_level_2'])
    # label_3 = resetLabelLv3(content[id]['label_level_3'])

    # if label_1 in label_1_dict:
    #     label_1_dict[label_1][0] += 1
    # else:
    #     label_1_dict[label_1] = [1, 0]

    if label_2 in label_2_dict:
        label_2_dict[label_2][0] += 1
    else:
        label_2_dict[label_2] = [1, 0]
    
    
    # predict_1 = model_label1.predict(text)[0][0]
    predict_2 = model_label2.predict(text)[0][0]
    # predict_3 = model_label3.predict(text)[0][0]

    # if(predict_1.replace('__label__', '') == label_1):
    #     true_label_1 += 1
    #     label_1_dict[label_1][1] += 1
    if(predict_2.replace('__label__', '') == label_2):
        true_label_2 += 1
        label_2_dict[label_2][1] += 1
    # if(predict_3.replace('__label__', '') == label_3):
    #     true_label_3 += 1
    total += 1
# print('label level 1 accurate: {}% ({}/{})'.format(true_label_1 * 100 / total, true_label_1, total))
print('tag level 2 accurate: {}% ({}/{})'.format(true_label_2 * 100 / total, true_label_2, total))
# print('tag level 3 accurate: {}% ({}/{})'.format(true_label_3 * 100 / total, true_lable_3, total))

label level 1 accurate: 92.70545264805635% (35534/38330)


In [6]:
for label in label_2_dict:
    print(label, label_2_dict[label][1], "/", label_2_dict[label][0])

荔湾区政府 35134 / 36018
市人力资源社会保障局 1 / 35
白云区政府 17 / 213
市卫生健康委 77 / 366
海珠区政府 5 / 148
市总工会 0 / 5
市邮政管理局 16 / 35
越秀集团 0 / 6
广州地铁集团 17 / 66
市排水公司 6 / 22
广州交投集团 0 / 4
珠江实业集团 11 / 25
越秀区政府 18 / 303
市医保局 2 / 21
番禺区政府 5 / 57
黄埔区政府 1 / 40
市住房城乡建设局 9 / 66
市公安局 70 / 150
广州供电局有限公司 40 / 67
12345热线管理机构 0 / 11
市国资委 2 / 5
市交通运输局 35 / 95
广州市自来水公司 31 / 116
广州市消费者委员会 0 / 2
穗康码工作专班 0 / 25
市规划和自然资源局 1 / 23
花都区政府 0 / 29
增城区政府 1 / 23
市教育局 8 / 25
南沙区政府 1 / 34
天河区政府 3 / 122
广州市公共交通集团有限公司 4 / 20
市政务服务数据管理局 0 / 3
广州市税务局 11 / 55
市民政局 0 / 11
市地方金融监管局 3 / 7
市商务局 0 / 3
市港务局 0 / 1
市消防支队 0 / 1
市燃气集团 0 / 6
市发展改革委 0 / 2
市残联 0 / 2
市财政局 0 / 1
市来穗人员服务管理局 3 / 8
市水投集团 0 / 2
市建筑集团 0 / 3
市城投集团 0 / 1
广州轻工工贸集团 1 / 2
市文化广电旅游局 0 / 2
中国电信广州分公司 0 / 5
市农业农村局 0 / 1
广州市广播电视台 0 / 1
广州环保投资集团 0 / 1
广州市烟草专卖局 0 / 3
农业银行广东省分行 0 / 2
市水务局 0 / 2
市林业园林局 1 / 4
中国广电广州网络股份有限公司 0 / 2
广州海事局 0 / 1
市城市管理综合执法局 0 / 6
广州交易集团有限公司（广州公共资源交易中心） 0 / 2
从化区政府 0 / 4
市政府外办 0 / 2
市新闻出版局（市版权局） 0 / 1
广州海关 0 / 1
中国移动广州分公司 0 / 1
市市场监管局 0 / 1
建设银行广东省分行 0 / 1
广州商贸投资控股集团有限

lr = 0.1 epoch = 25

标签数量

In [8]:
print('tag 1 num: ', len(model_label1.labels))
# print('tag 2 num: ', len(model_label2.labels))
# print('tag 3 num: ', len(model_label3.labels))

tag 2 num:  459
tag 3 num:  970
